# SYCL Migration - Matrix Multiplication cuBlas

##### Sections
- [Introduction](#Introduction)
- [Analyze CUDA source](#Analyze-CUDA-source)
- [Migrate CUDA source to SYCL source](#Migrate-CUDA-source-to-SYCL-source)
- [Analyze, Compile and Run the migrated SYCL source](#Analyze,-Compile-and-Run-the-migrated-SYCL-source)
- [Source Code](#Source-Code)

## Learning Objectives
* Use SYCLomatic Tool to migrate a simple single source CUDA application
* Use various command line options of `SYCLomatic` for CUDA to SYCL migration
* Compile and run migrated SYCL code on Intel CPUs and GPUs
* Optimize the migrated SYCL code with manual coding

## Introduction

This module will walk you through migrating CUDA code to SYCL code using Intel SYCLomatic Tool

#### Requirements
1. NVidia CUDA development machine
2. Development machine with Intel CPU/GPU OR a Intel Developer Cloud account

#### Migration Process
We will do the following steps in this hands-on workshop:
- Analyze CUDA source
- Migrate CUDA source to SYCL source
- Analyze, Compile and Run the migrated SYCL source

## Analyze CUDA source

The CUDA source for "Matrix Multiplication cuBlas" example is available on [Nvidia Github](https://github.com/NVIDIA/cuda-samples/tree/master/Samples/4_CUDA_Libraries/matrixMulCUBLAS/)

Pull the entire repository on your CUDA Development machine.

```
git clone https://github.com/NVIDIA/cuda-samples.git

cd cuda-samples/Samples/4_CUDA_Libraries/matrixMulCUBLAS/
```

The CUDA source for the Matrix Multiplication cuBlas implementation is in the following files.

[__matrixMulCUBLAS.cpp__](https://github.com/NVIDIA/cuda-samples/blob/master/Samples/4_CUDA_Libraries/matrixMulCUBLAS/) — host code for:
- setting up CUDA stream
- memory allocation on GPU
- initialization of data on CPU
- copy data to GPU memory for computation
- kickoff computation on GPU using cuBlas function
- verify and print results


## Migrate CUDA source to SYCL source

<p style="background-color:#cdc"> Note: A CUDA development machine is required to accomplish the task in this section </p>

Now that we have analyzed the CUDA source, we will migrate the CUDA source into SYCL source using the __SYCLomatic Tool__.

In this exercise, we will walk you through step-by-step to migrate the CUDA code.

#### Requirements

Make sure you have a __NVIDIA CUDA development machine__ that can __compile and run CUDA code__. The next step is to install the tools for migrating CUDA to SYCL:

- Install SYCLomatic Tool on this machine
  - go to https://github.com/oneapi-src/SYCLomatic/releases/
  - copy link to latest `linux_release.tgz` from assets
  - on the CUDA development machine: `mkdir syclomatic; cd syclomatic`
  - `wget <link to linux_release.tgz>`
  - `tar -xvf linux_release.tgz`
  - `export PATH="/home/$USER/syclomatic/bin:$PATH"`
  - Verify installation: `c2s --version`
- pull the CUDA samples repo to this machine
  - `git clone https://github.com/NVIDIA/cuda-samples.git`
- Compile and run the `matrixMulCUBLAS` sample
  - `cd cuda-samples/Samples/4_CUDA_Libraries/matrixMulCUBLAS`
  - `make`


### Migrate CUDA source to SYCL source using SYCLomatic

On the NVIDIA CUDA Development machine, go to the CUDA source folder and generate a compilation database with the tool `intercept-build`. This creates a JSON file with all the compiler invocations, stores the names of the input files and the compiler options.

```
make clean
intercept-build make
```

This will create a file named `compile_commands.json` in the sample folder.

Next, use the SYCLomatic Tool (c2s) to migrate the code; it will store the result in the migration folder `dpct_output`:

```
c2s -p compile_commands.json --in-root ../../.. --use-custom-helper=api
```

The `--use-custom-helper=api` option will copy the SYCLomatic helper header files to output directory.

The `--in-root` option will specify the path for all the common include files for the CUDA project.

This command should migrate the CUDA source to the C++ SYCL source in a folder named `dpct_output` by default, and the folder will have the C++ SYCL source along with any dependencies from the `Common` folder:

- `matrixMulCUBLAS.cpp.dp.cpp`

This command may also throw a bunch of warnings about the migration process. The CUDA code that cannot be automatically migrated will have warning comments generated in the migrated source files, which have to be manually migrated.


## Analyze, Compile and Run the migrated SYCL source

<p style="background-color:#cdc"> Note: The tasks in this section should be done on Intel DevCloud or on a system with oneAPI Base toolkit installed.</p>

The migrated SYCL code are in the `Samples` folder under the `dpct_output` folder:
- `matrixMulCUBLAS.cpp.dp.cpp`

The `dpct_output` folder also has headers files needed for compiling the migrated SYCL code. The `Common` folder has header files with CUDA helper functions which are migrated to SYCL and the `include` folder has header files with SYCLomatic helper functions.

#### Requirements

Make sure you have one of the following:
- __Development machine with Intel CPU/GPU__ with Intel oneAPI Base Toolkit installed
- __Intel Developer Cloud__ account to access the Intel CPUs/GPUs on the cloud

### Compiling migrated SYCL code

Copy the files mentioned above in `dpct_output` folder on __Nvidia Development Machine__ to __Intel Developer Cloud__

To compile the migrated SYCL code we can use the following command:
```
icpx -fsycl -I ../../../Common -I ../../../include *.cpp -lmkl_sycl -lmkl_intel_ilp64 -lmkl_sequential -lmkl_core
```

There may be compile errors based on whether all of the CUDA code was migrated to SYCL or not. The migrated code may also include comments with warning messages, which could help make it easier to fix the errors. These errors have to be manually fixed to get the code to compile.


### SYCLomatic warnings in migrated SYCL code

There are other migration warnings that can be looked at to see if any improvements can be made, below are couple examples of warning in `matrixMulCUBLAS.cpp.dp.cpp`:


```cpp
         /*
         DPCT1005:27: The SYCL device version is different from CUDA Compute
         Compatibility. You may need to rewrite this code.
         */
      deviceProp.get_name(), deviceProp.get_major_version(),
         deviceProp.get_minor_version()
```


```cpp
    /*
    DPCT1012:38: Detected kernel execution time measurement pattern and
    generated an initial code for time measurements in SYCL. You can change the
    way time is measured depending on your goals.
    */
    /*
    DPCT1024:39: The original code returned the error code that was further
    consumed by the program logic. This original code was replaced with 0. You
    may need to rewrite the program logic consuming the error code.
    */
    start_ct1 = std::chrono::steady_clock::now();
```

### Compile and Run the migrated SYCL source

Once you have successfully migrated the CUDA source to the SYCL source, verify that the migrated SYCL code is functioning correctly by compiling and running it on the Intel Developer Cloud, which has a variety of Intel CPUs and GPUs available for development.

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! ./q.sh run_sycl_migrated.sh gen9

### SYCL Code Migration Analysis

When comparing the CUDA code and migrated SYCL code, we can see that there are some 1:1 equivalent calls, which are listed below in the table:

| Functionality|CUDA|SYCL
|-|-|-
| header file|`#include <cuda_runtime.h>`|`#include <CL/sycl.hpp>` <br> `#include <dpct/dpct.hpp>`
|library header file|`#include <cublas_v2.h>`|`#include <dpct/blas_utils.hpp>`
| Memory allocation on device| `cudaMalloc((void **)&d_A, mem_size_A)`| `d_A = (float *)sycl::malloc_device(mem_size_A, dpct::get_default_queue())`
| Copy memory between host and device| `cudaMemcpy(d_A, h_A, mem_size_A, cudaMemcpyHostToDevice)`| `dpct::get_default_queue().memcpy(d_A, h_A, mem_size_A).wait()`
| Free device memory allocation| `free(h_A)` | `sycl::free(h_A, q)`
| gemm function | `cublasSgemm(...)`| `oneapi::mkl::blas::column_major::gemm(...)`


The main kernel operation to perform matrix multiplication is done using `cuBlas` library in CUDA which is migrated to `oneapi::mkl` library function as shown below:

cuBlas sgemm function to perform operation in CUDA code:
```cpp
      checkCudaErrors(cublasSgemm(
          handle, CUBLAS_OP_N, CUBLAS_OP_N, matrix_size.uiWB, matrix_size.uiHA,
          matrix_size.uiWA, &alpha, d_B, matrix_size.uiWB, d_A,
          matrix_size.uiWA, &beta, d_C, matrix_size.uiWB));
```

oneapi::mkl::blas gemm function to perform operation in SYCL code:
```cpp
      checkCudaErrors(
          (oneapi::mkl::blas::column_major::gemm(
               *handle, oneapi::mkl::transpose::nontrans,
               oneapi::mkl::transpose::nontrans, matrix_size.uiWB,
               matrix_size.uiHA, matrix_size.uiWA, alpha, d_B, matrix_size.uiWB,
               d_A, matrix_size.uiWA, beta, d_C, matrix_size.uiWB),
           0));
```

## Source Code

This section describes the location of the CUDA source and the contents of different SYCL source code directories in this project.

| folder name | source code description
| --- | ---
| [CUDA github](https://github.com/NVIDIA/cuda-samples/tree/master/Samples/4_CUDA_Libraries/matrixMulCUBLAS) | Original CUDA Source used for migration
| dpct_output | SYCL migration output from SYCLomatic Tool, compiles without errors
| sycl_migrated | Same as dpct_output, compiles without errors

## Summary

In this module we have learnt how to migrate simple CUDA source to SYCL source to get functionality using `SYCLomatic` and then analized/optimized the SYCL source by manually coding. 